In [14]:
from models import *
from train import*

import
import


In [68]:
class Options:
    def __init__(self):
        self.n_epochs = 501
        self.batch_size = 32
        self.d_lr = 0.0001
        self.q_lr = 0.0001
        self.g_lr = 0.0001
        self.b1 = 0.5
        self.b2 = 0.999
        self.latent_dim = 512
        self.model_save_dir = './model_cwgan_mgmno/'
        self.load_model = False
        self.load_generator = None
        self.load_discriminator = None
        self.load_q = None
        self.constraint_epoch = 10000
        self.gen_dir = './gen_image_cwgan_mgmno/'
        self.trainingdata = 'mgmno_1000.pickle'
        self.input_dim = 512+28+1

opt = Options()
print(opt)

In [69]:
import torch

In [70]:
generator = Generator(opt)
discriminator = Discriminator(opt)
net_Q = QHead_(opt)

In [71]:
device = torch.device('cpu')
generator.load_state_dict(torch.load('/Users/oridwan/Desktop/generator_500', 
                                     map_location=device))
discriminator.load_state_dict(torch.load('/Users/oridwan/Desktop/discriminator_500', 
                                     map_location=device))
net_Q.load_state_dict(torch.load('/Users/oridwan/Desktop/Q_500', 
                                     map_location=device))

<All keys matched successfully>

In [119]:
import numpy as np
#from view_atoms_mgmno import view_atoms_classifier
import numpy as np
from ase import Atoms
from ase.io import read,write
import sys
import torch

def view_atoms_classifier(image,mg_index,mn_index, o_index, view=True):
    
	x= image.reshape(-1,3)
	mg = x[2:10,:]
	mn = x[10:18,:]
	o = x[18:,:]

	l = x[0,:]*30
	a = x[1,:]*180
	c = np.hstack((l,a))
	atoms = Atoms('H')
	atoms.set_cell(c)
	cell = atoms.get_cell()
	t = np.isnan(cell)
	tt = np.sum(t)
	isnan = False
	if not tt == 0:
		isnan = True
		print(cell)
		print(l)
		print(a)
	#_,mg_index = torch.max(mg_label,dim=1)
	#_,mn_index = torch.max(mn_label,dim=1)
	#_,o_index = torch.max(o_label,dim=1)
	
	#mg_index = mg_index.reshape(8,).detach().cpu().numpy()
	#mn_index = mn_index.reshape(8,).detach().cpu().numpy()
	#o_index = o_index.reshape(12,).detach().cpu().numpy()
	
	
	mg_pos = mg[np.where(mg_index)]
	mn_pos = mn[np.where(mn_index)]
	o_pos = o[np.where(o_index)]
	
	n_mg = len(mg_pos)
	n_mn = len(mn_pos)
	n_o = len(o_pos)
	
	if n_mg == 0:
		mg_pos = np.array([0.1667,0.1667,0.1667]).reshape(1,3)
		n_mg = 1
	if n_mn == 0:
		mn_pos = np.array([0.1667,0.1667,0.1667]).reshape(1,3)
		n_mn = 1
		
	if n_o == 0:
		o_pos = np.array([0.1667,0.1667,0.1667]).reshape(1,3)
		n_o = 1

	pos = np.vstack((mg_pos,mn_pos,o_pos))
	scaled_pos = back_to_10_cell(pos,n_mg,n_mn,n_o)
	atoms = back_to_real_cell(scaled_pos, cell, n_mg,n_mn,n_o)
	atoms.set_pbc([1,1,1])
	if view :
		atoms.edit()
			
	return atoms, x, isnan

In [115]:
FloatTensor = torch.FloatTensor

fake_c_mg_int = np.random.randint(0, 8, opt.batch_size)
print("fake_c_mg_int",fake_c_mg_int)
fake_c_mg = to_categorical(fake_c_mg_int,num_columns = 8)
print("fake_c_mg",fake_c_mg)
fake_c_mn_int = np.random.randint(0, 8, opt.batch_size)
print("fake_c_mn_int",fake_c_mn_int)

fake_c_mn = to_categorical(fake_c_mn_int,num_columns = 8)
print("fake_c_mn",fake_c_mn)

fake_c_o_int = np.random.randint(0,12,opt.batch_size)
print("fake_c_o_int",fake_c_o_int)

fake_c_o = to_categorical(fake_c_o_int, num_columns = 12)
print("fake_c_o",fake_c_o)


natoms_fake = fake_c_mg_int + fake_c_mn_int + fake_c_o_int + 3
natoms_fake = Variable(FloatTensor(natoms_fake)/(28.0)).unsqueeze(-1)
z = FloatTensor(np.random.normal(0,1,(opt.batch_size, opt.latent_dim)))


fake_c_mg_int [4 0 2 2 3 5 4 4 0 1 4 3 6 2 6 1 2 1 6 0 3 7 2 0 2 2 2 3 5 2 5 5]
fake_c_mg tensor([[0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.,

In [125]:
fake = generator(z,fake_c_mg,fake_c_mn,fake_c_o,natoms_fake)
print("++++++++++++++++++++++++++",fake.shape)
fake_feature, G = discriminator(fake)
fake_mg_label, fake_mn_label, fake_o_label, fake_mg_cat, fake_mn_cat, fake_o_cat, fake_cell_pred = net_Q(fake)

gen_input torch.Size([32, 541])
l1 layer in generator torch.Size([32, 3584])
Convert it to pixel data torch.Size([32, 128, 28, 1])
Conv1 layer in generator torch.Size([32, 256, 28, 3])
Conv2 layer in generator torch.Size([32, 512, 28, 3])
Conv3 layer in generator torch.Size([32, 256, 28, 3])
Conv4 layer in generator torch.Size([32, 1, 28, 3])
flatten Conv4 layer in generator torch.Size([32, 84])
Apply activation to get pos torch.Size([32, 1, 28, 3])
Get Cell from position torch.Size([32, 6])
Final generator output torch.Size([32, 1, 30, 3])
++++++++++++++++++++++++++ torch.Size([32, 1, 30, 3])
model layer in Discriminator torch.Size([32, 256, 30, 1])
extract cell torch.Size([32, 256, 2, 1])
2D average pooling on Mg torch.Size([32, 256, 1, 1])
2D average pooling on Mn torch.Size([32, 256, 1, 1])
2D average pooling on O torch.Size([32, 256, 1, 1])
final output torch.Size([32, 1280])
Feature layer in Discriminator torch.Size([32, 200])
Output layer in Discriminator torch.Size([32, 10])
In

In [127]:
fake_mg_label.shape

torch.Size([32, 2, 8, 1])

In [131]:
_,mg_index = torch.max(fake_mg_label,dim=1)
_,mn_index = torch.max(fake_mn_label,dim=1)
_,o_index = torch.max(fake_o_label,dim=1)
print(mg_index[0, :, 0])
print(mn_index[0, :, 0])
print(o_index[0, :, 0])
mg_index[0] 

tensor([1, 0, 0, 0, 1, 1, 1, 1])
tensor([0, 1, 0, 1, 1, 1, 1, 0])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1])


tensor([[1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1]])

In [132]:
for i in range(opt.batch_size):
    atom, _, _ = view_atoms_classifier(fake[i].detach().numpy(),
                                  mg_index[i, :, 0].numpy(),
                                  mn_index[i, :, 0].numpy(),
                                  o_index[i, :, 0].numpy(),
                                  view=False)
    #print(i, mg_index[i])
    #atom.write(str(i)+'.cif', format='cif')
    print(i, atom)

0 Atoms(symbols='Mg5Mn5O10', pbc=True, cell=[[6.449182510375977, 0.0, 0.0], [-1.2519056855024007, 7.1369486579376025, 0.0], [-1.5252379238172074, -1.4272080458662846, 7.636989493824114]])
1 Atoms(symbols='MgMnO2', pbc=True, cell=[[3.65177845954895, 0.0, 0.0], [0.8182627211507978, 3.0294424592056943, 0.0], [0.4711042184319994, 0.3897783521049263, 4.451607840354251]])
2 Atoms(symbols='Mg4Mn5O12', pbc=True, cell=[[7.209471225738525, 0.0, 0.0], [2.169378406785703, 6.830359125680276, 0.0], [5.039673943532296, -1.4094761180995057, 7.368873908735248]])
3 Atoms(symbols='Mg3Mn4O8', pbc=True, cell=[[5.614352703094482, 0.0, 0.0], [3.487483742188644, 5.029972431654403, 0.0], [5.544984583296392, 2.273897010008461, 4.8450441352583224]])
4 Atoms(symbols='Mg3Mn8O10', pbc=True, cell=[[10.235210418701172, 0.0, 0.0], [-3.521708976333643, 5.285621254172655, 0.0], [-0.9810748264793837, -0.256905368188995, 8.912025675693277]])
5 Atoms(symbols='Mg5MnO5', pbc=True, cell=[[2.9878625869750977, 0.0, 0.0], [0.666

In [81]:
_,mn_index = torch.max(fake_mn_label,dim=1)
print(fake_mn_label.shape)

torch.Size([32, 2, 8, 1])


In [90]:
mn_index[0, :, 0].shape

torch.Size([8])

In [80]:
fake_mn_label.shape

torch.Size([32, 2, 8, 1])